<center><h1> Data Selection</h1></center>
***


Here we will see, which dataset is more biased, or gives more mean squared error with same model. First I need to import libraries I need to check with which dataset my models perform better. To check I will use a technique called cross-validation. I will perform prediction on three machine learning algorithms. And then I import datasets I need.  
Also we need our dataset to be shuffled. So far I made three different datasets. They will be shaffled, in script. Third dataset is mix of first and second dataset. Some values are dropped, some are imputed with another categorical values, and "Пробег" column is the only column subjected to predictive imputation, the reason I created the third dataset was in hope to maximally reduce the dataset bias

In [3]:
import pandas as pd 
import pickle 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import set_jupyter_path

In [4]:
import warnings
from src.car_price_prediction.utils import dataset_manager, df_utils
data = dataset_manager.get_cleaned_outliers_dataset()
from notebook_helper_functions.data_impute import get_imputed_dataset

def make_imputed_dataset(data):
    data_for_imp = data.copy()
    #erroneous warining inside imported library
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        data_imp = get_imputed_dataset(data_for_imp)
    data_for_imp = shuffle(data_for_imp)
    return data_imp

def get_dropped_dataset(data):
    data_for_drop = data.copy()
    data_for_drop = data_for_drop.dropna()
    data_for_drop = shuffle(data_for_drop)
    return data_for_drop

def get_max_unbiased_dataset():
    return dataset_manager.get_processed_dataset()

data_imputed = make_imputed_dataset(data)
data_dropped = get_dropped_dataset(data)
data_max_unbiased = get_max_unbiased_dataset()

First I will divide the data into dataset and target variables and then I create predictor class instances and I will use cross-validation with 10 folds.

In [5]:
from src.car_price_prediction.utils import df_utils

def get_X_y(data):
    X,y = df_utils.get_data_and_target(data)
    X = pd.get_dummies(X)
    return X,y

X_imp, y_imp = get_X_y(data_imputed)
X_drop, y_drop = get_X_y(data_dropped)
X_unbias,y_unbias = get_X_y(data_max_unbiased)

Cv argument means how many folds I want to use in cross-validation, more folds mean more unbiased error, for forest I use 5, because it's computationally more expensive.

In [6]:
def print_cv_score(estimator, est_name, cv,scoring = "neg_mean_absolute_error"):
    score_imp = cross_val_score(estimator = estimator,X = X_imp,y = y_imp,cv = cv,scoring = scoring) 
    score_drop = cross_val_score(estimator=estimator,X = X_drop,y = y_drop,cv = cv,scoring = scoring) 
    score_unbias = cross_val_score(estimator=estimator, X = X_unbias, y = y_unbias, cv = cv, scoring = scoring)
    print("MSE for imputed data for %s: \t%.3f" % (est_name,score_imp.mean()))
    print("MSE for dropped data for %s: \t%.3f" % (est_name,score_drop.mean()))
    print("MSE for unbiased data for %s: \t%.3f" % (est_name,score_unbias.mean()))
 

Here, we can see how Linear Regression manages to predict on given datasets

In [7]:
print_cv_score(estimator = LinearRegression(),est_name = "Linear Regression",cv = 3) 

MSE for imputed data for Linear Regression: 	-6714.491
MSE for dropped data for Linear Regression: 	-7794.199
MSE for unbiased data for Linear Regression: 	-6647.573


Now, I will use K-Nearest, with standartization, because without standartization, knn will not work properly

In [8]:
scalar = StandardScaler()
knn = KNeighborsRegressor()
pipeline_knn = Pipeline([("sc",scalar),('estimator',knn)])
print_cv_score(pipeline_knn,est_name = "KNearestNeighbors",cv = 3) 

MSE for imputed data for KNearestNeighbors: 	-5253.894
MSE for dropped data for KNearestNeighbors: 	-5798.370
MSE for unbiased data for KNearestNeighbors: 	-4540.195


And the last one I am going to use is Random Forest Regressor, it does not need any standartization

In [9]:
scalar = StandardScaler()
forest = RandomForestRegressor()
print_cv_score(forest,est_name="Random Forest",cv = 3)

MSE for imputed data for Random Forest: 	-2569.833
MSE for dropped data for Random Forest: 	-2794.853
MSE for unbiased data for Random Forest: 	-2247.920


We can see that the most biased dataset is dropped dataset, so we will not use it like our processed data, and also we can see that imputed dataset performs slightly worse than maximum unbiased dataset, it's due to the bias in imputed dataset, in next notebook, I will confirm, that the best dataset to use is indeed max_unbiased dateset, and I will prove it in completely new and unseen data.